In [38]:
from urllib.request import urlopen
import os
from dotenv import load_dotenv
import json
import pandas as pd
import requests
from IPython.display import display

In [39]:
if __name__ == "__main__":
    load_dotenv()
    api_key = os.environ.get("api_key")
    params = {
        "frequency": "monthly",
        "start": "2024-01",
        "end":"2024-02",
        "sort[0][column]": "period",
        "offset": "0",
        "length": "5000",
        "api_key": api_key
    } 

In [40]:
# Set the URL
              
url = f"https://api.eia.gov/v2/petroleum/pri/spt/data/?data[0]=value&sort[0][direction]=desc"

response = requests.get(
    url=url, params=params
)

response_data = response.json()

df_petrolium = pd.json_normalize(data=response_data)
df_petrolium =pd.DataFrame(df_petrolium["response.data"][0])
# pd.options.display.max_columns = None
# df_petrolium.head()
df_petrolium

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2024-01,Y05LA,LOS ANGELES,EPMRR,Reformulated Regular Gasoline,PF4,Spot Price FOB,EER_EPMRR_PF4_Y05LA_DPG,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL
1,2024-01,Y35NY,NEW YORK CITY,EPD2DXL0,No 2 Diesel Low Sulfur (0-15 ppm),PF4,Spot Price FOB,EER_EPD2DXL0_PF4_Y35NY_DPG,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL
2,2024-01,Y44MB,NA,EPLLPA,Propane,PF4,Spot Price FOB,EER_EPLLPA_PF4_Y44MB_DPG,"Mont Belvieu, TX Propane Spot Price FOB (Dolla...",.821,$/GAL
3,2024-01,RGC,NA,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_RGC_DPG,U.S. Gulf Coast Conventional Gasoline Regular ...,2.146,$/GAL
4,2024-01,Y35NY,NEW YORK CITY,EPMRU,Conventional Regular Gasoline,PF4,Spot Price FOB,EER_EPMRU_PF4_Y35NY_DPG,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL
5,2024-01,YCUOK,NA,EPCWTI,WTI Crude Oil,PF4,Spot Price FOB,RWTC,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",74.15,$/BBL
6,2024-01,Y35NY,NEW YORK CITY,EPD2F,No 2 Fuel Oil / Heating Oil,PF4,Spot Price FOB,EER_EPD2F_PF4_Y35NY_DPG,New York Harbor No. 2 Heating Oil Spot Price F...,2.59,$/GAL
7,2024-01,RGC,NA,EPJK,Kerosene-Type Jet Fuel,PF4,Spot Price FOB,EER_EPJK_PF4_RGC_DPG,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Pr...,2.585,$/GAL
8,2024-01,Y05LA,LOS ANGELES,EPD2DC,Carb Diesel,PF4,Spot Price FOB,EER_EPD2DC_PF4_Y05LA_DPG,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL
9,2024-01,RGC,NA,EPD2DXL0,No 2 Diesel Low Sulfur (0-15 ppm),PF4,Spot Price FOB,EER_EPD2DXL0_PF4_RGC_DPG,U.S. Gulf Coast Ultra-Low Sulfur No 2 Diesel S...,2.569,$/GAL


In [41]:
df_petrolium.columns

Index(['period', 'duoarea', 'area-name', 'product', 'product-name', 'process',
       'process-name', 'series', 'series-description', 'value', 'units'],
      dtype='object')

In [42]:
#Select Needed Columns

selected_petroleum = df_petrolium[["period", "area-name", "product-name", "process-name", "series-description", "value", "units"]]
selected_petroleum

,period,area-name,product-name,process-name,series-description,value,units
0,2024-01,LOS ANGELES,Reformulated Regular Gasoline,Spot Price FOB,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL
1,2024-01,NEW YORK CITY,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL
2,2024-01,NA,Propane,Spot Price FOB,"Mont Belvieu, TX Propane Spot Price FOB (Dolla...",.821,$/GAL
3,2024-01,NA,Conventional Regular Gasoline,Spot Price FOB,U.S. Gulf Coast Conventional Gasoline Regular ...,2.146,$/GAL
4,2024-01,NEW YORK CITY,Conventional Regular Gasoline,Spot Price FOB,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL
5,2024-01,NA,WTI Crude Oil,Spot Price FOB,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",74.15,$/BBL
6,2024-01,NEW YORK CITY,No 2 Fuel Oil / Heating Oil,Spot Price FOB,New York Harbor No. 2 Heating Oil Spot Price F...,2.59,$/GAL
7,2024-01,NA,Kerosene-Type Jet Fuel,Spot Price FOB,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Pr...,2.585,$/GAL
8,2024-01,LOS ANGELES,Carb Diesel,Spot Price FOB,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL
9,2024-01,NA,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,U.S. Gulf Coast Ultra-Low Sulfur No 2 Diesel S...,2.569,$/GAL


In [43]:
#Transformation 1 -> Renaming

renamed_petroleum = selected_petroleum.rename(columns={
    "area-name": "city",
    "product-name": "product",
    "series-description": "series",
    "process-name": "process"
})
renamed_petroleum

,period,city,product,process,series,value,units
0,2024-01,LOS ANGELES,Reformulated Regular Gasoline,Spot Price FOB,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL
1,2024-01,NEW YORK CITY,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL
2,2024-01,NA,Propane,Spot Price FOB,"Mont Belvieu, TX Propane Spot Price FOB (Dolla...",.821,$/GAL
3,2024-01,NA,Conventional Regular Gasoline,Spot Price FOB,U.S. Gulf Coast Conventional Gasoline Regular ...,2.146,$/GAL
4,2024-01,NEW YORK CITY,Conventional Regular Gasoline,Spot Price FOB,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL
5,2024-01,NA,WTI Crude Oil,Spot Price FOB,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",74.15,$/BBL
6,2024-01,NEW YORK CITY,No 2 Fuel Oil / Heating Oil,Spot Price FOB,New York Harbor No. 2 Heating Oil Spot Price F...,2.59,$/GAL
7,2024-01,NA,Kerosene-Type Jet Fuel,Spot Price FOB,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Pr...,2.585,$/GAL
8,2024-01,LOS ANGELES,Carb Diesel,Spot Price FOB,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL
9,2024-01,NA,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,U.S. Gulf Coast Ultra-Low Sulfur No 2 Diesel S...,2.569,$/GAL


In [44]:
df = pd.DataFrame(renamed_petroleum)

# Transformation 2 -> Datatype Casting

df['value'] = df['value'].astype(float)
print(df.dtypes)

period      object
city        object
product     object
process     object
series      object
value      float64
units       object
dtype: object


In [45]:
# Transformation 3 -> Sorting

sorted_petroleum = df.sort_values(by='value', ascending=False)
sorted_petroleum

,period,city,product,process,series,value,units
10,2024-01,NA,UK Brent Crude Oil,Spot Price FOB,Europe Brent Spot Price FOB (Dollars per Barrel),80.120,$/BBL
5,2024-01,NA,WTI Crude Oil,Spot Price FOB,"Cushing, OK WTI Spot Price FOB (Dollars per Ba...",74.150,$/BBL
1,2024-01,NEW YORK CITY,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL
6,2024-01,NEW YORK CITY,No 2 Fuel Oil / Heating Oil,Spot Price FOB,New York Harbor No. 2 Heating Oil Spot Price F...,2.590,$/GAL
7,2024-01,NA,Kerosene-Type Jet Fuel,Spot Price FOB,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Pr...,2.585,$/GAL
8,2024-01,LOS ANGELES,Carb Diesel,Spot Price FOB,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL
9,2024-01,NA,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,U.S. Gulf Coast Ultra-Low Sulfur No 2 Diesel S...,2.569,$/GAL
0,2024-01,LOS ANGELES,Reformulated Regular Gasoline,Spot Price FOB,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL
4,2024-01,NEW YORK CITY,Conventional Regular Gasoline,Spot Price FOB,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL
3,2024-01,NA,Conventional Regular Gasoline,Spot Price FOB,U.S. Gulf Coast Conventional Gasoline Regular ...,2.146,$/GAL


In [46]:
# Transformation 4 -> Filtering

filtered_petroleum = df[df['city'] != 'NA']
filtered_petroleum

,period,city,product,process,series,value,units
0,2024-01,LOS ANGELES,Reformulated Regular Gasoline,Spot Price FOB,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL
1,2024-01,NEW YORK CITY,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL
4,2024-01,NEW YORK CITY,Conventional Regular Gasoline,Spot Price FOB,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL
6,2024-01,NEW YORK CITY,No 2 Fuel Oil / Heating Oil,Spot Price FOB,New York Harbor No. 2 Heating Oil Spot Price F...,2.590,$/GAL
8,2024-01,LOS ANGELES,Carb Diesel,Spot Price FOB,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL


In [47]:
# Transformation 5 -> Merge

countries = pd.read_csv("etl_project/country.csv")
merged_petroleum = pd.merge(left=df, right=countries, on=["city"])
merged_petroleum

,period,city,product,process,series,value,units,country
0,2024-01,LOS ANGELES,Reformulated Regular Gasoline,Spot Price FOB,Los Angeles Reformulated RBOB Regular Gasoline...,2.274,$/GAL,america
1,2024-01,LOS ANGELES,Carb Diesel,Spot Price FOB,"Los Angeles, CA Ultra-Low Sulfur CARB Diesel S...",2.573,$/GAL,america
2,2024-01,NEW YORK CITY,No 2 Diesel Low Sulfur (0-15 ppm),Spot Price FOB,New York Harbor Ultra-Low Sulfur No 2 Diesel S...,2.681,$/GAL,america
3,2024-01,NEW YORK CITY,Conventional Regular Gasoline,Spot Price FOB,New York Harbor Conventional Gasoline Regular ...,2.244,$/GAL,america
4,2024-01,NEW YORK CITY,No 2 Fuel Oil / Heating Oil,Spot Price FOB,New York Harbor No. 2 Heating Oil Spot Price F...,2.590,$/GAL,america


In [48]:
#Transformation 6 & 7 -> Multiple Grouping + Aggregation

merged_petroleum.groupby(["city", "product", "units"]).agg({
    "value": "mean",
}).sort_values(
    by='city', 
    ascending=False
    )

value
city          product                           units       
NEW YORK CITY Conventional Regular Gasoline     $/GAL  2.244
              No 2 Diesel Low Sulfur (0-15 ppm) $/GAL  2.681
              No 2 Fuel Oil / Heating Oil       $/GAL  2.590
LOS ANGELES   Carb Diesel                       $/GAL  2.573
              Reformulated Regular Gasoline     $/GAL  2.274